In [48]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from scipy.stats import randint, uniform

# Definir ruta del dataset
CSV_PATH = "ventas_2025-10-02.csv"  
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Parámetros dinámicos para el churn
ALPHA = 3.0
MIN_DAYS = 15
MAX_DAYS = 180

# Cargar dataset
df = pd.read_csv(CSV_PATH)

# Combinar fecha y hora en una sola columna de tipo datetime
date = pd.to_datetime(df["CreadoEl"], dayfirst=True, errors="coerce")
hora = df["Hora"].astype(str).str.strip()
hora = np.where(hora.str.match(r"^\d{1,2}:\d{2}$"), hora + ":00", hora)
time_delta = pd.to_timedelta(hora, errors="coerce")
df["dt"] = date + time_delta.fillna(pd.Timedelta(0))

# Limpiar valores de importe
def to_number_vec(s):
    s = s.astype(str).str.replace(r"[.\s$ARSa-zA-Z]", "", regex=True)
    s = s.str.replace(",", ".", regex=False)
    return pd.to_numeric(s, errors="coerce")

df["ValorNeto_num"] = to_number_vec(df["ValorNeto"])

# Definir columna de cliente
key_col = "Cliente" if "Cliente" in df.columns else "CUIT"

# Ordenar por tiempo
df = df.sort_values([key_col, "dt"]).reset_index(drop=True)
cutoff_dt = df["dt"].max() # La fecha de corte es la fecha de la ultima venta en el dataset


/var/folders/y0/404pxmfn357ftb9snt8c2x4r0000gn/T/ipykernel_98076/3333807376.py:27: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  date = pd.to_datetime(df["CreadoEl"], dayfirst=True, errors="coerce")


In [49]:
# Diferencia en días entre compras por cliente y se hace un promedio
df["diff_days"] = df.groupby(key_col)["dt"].diff().dt.days
mean_gap = df.groupby(key_col)["diff_days"].mean().rename("mean_gap")
global_median_gap = df["diff_days"].dropna().median()
mean_gap = mean_gap.fillna(global_median_gap)

# Umbral dinámico: multiplica su frecuencia normal por ALPHA, limitado por MIN_DAYS y MAX_DAYS
dyn_threshold = (ALPHA * mean_gap).clip(lower=MIN_DAYS, upper=MAX_DAYS).rename("dyn_threshold")

# Días desde la última compra: buscamos ultima fecha de compra por cliente y calculamos diferencia con fecha de corte (esto sería la recencia)
last_purchase = df.groupby(key_col)["dt"].max().rename("last_purchase_dt")
days_since_last = (cutoff_dt - last_purchase).dt.days.rename("days_since_last")

# Etiqueta churn: 1 si superó el umbral dinámico. Variable objetivo.
label = (days_since_last > dyn_threshold).astype(int).rename("churn")


In [51]:
# CHURN.IPYNB - CELDA 37 (CÓDIGO MODIFICADO Y CORREGIDO)

import pandas as pd
import numpy as np
# Asumiendo que key_col, cutoff_dt, df están definidos en celdas previas

# --- 1. Función para calcular la Moda ---
def mode_agg(x):
    """Calcula el valor más frecuente (moda), ignorando NaNs."""
    return x.mode()[0] if not x.mode().empty else np.nan

# --- 2. Definición del diccionario de agregación (incluye Moda) ---
agg_dict = {
    "Documento": "count",
    "ValorNeto_num": ["sum", "mean", "std", "median"],
    "Vendedor": pd.Series.nunique,
    "Oficina": pd.Series.nunique,
    "Localidad": [pd.Series.nunique, mode_agg], # <-- AÑADIDO: Moda de Localidad
    "Nombre1": [pd.Series.nunique, mode_agg],   # <-- AÑADIDO: Moda de Nombre1
    "dt": "min" # Se agrega la fecha mínima para antigüedad
}

# --- 3. Agregación y renombramiento de columnas ---
features = df.groupby(key_col).agg(agg_dict)

# Aplanar el multi-índice de columnas
features.columns = [
    "_".join(c).replace("Documento_count", "n_purchases") 
    if isinstance(c, tuple) else c 
    for c in features.columns
]

# Renombrar las nuevas columnas de moda para claridad
features = features.rename(columns={
    'Localidad_mode_agg': 'Localidad_Moda',
    'Nombre1_mode_agg': 'Nombre1_Moda',
    'dt_min': 'first_purchase_dt' # Renombrar la primera fecha de compra
})
# Las columnas nunique también se renombran si se aplicaron múltiples agregaciones
# Ej: 'Localidad_nunique'

# --- 4. Concatenación con variables de tiempo y Churn (CORRECCIÓN CLAVE) ---
# Usamos las Series 'mean_gap', 'dyn_threshold', 'days_since_last', y 'label' directamente.
# El error 'df_model is not defined' se corrige usando las variables del notebook.
cust_df = pd.concat([
    features, 
    mean_gap, 
    dyn_threshold, 
    last_purchase, 
    days_since_last, 
    label
], axis=1)

# Se elimina la línea de join de first_purchase porque ya está en 'features'

# --- 5. Agregar features adicionales (Antigüedad y Frecuencia Normalizada) ---
cust_df["tenure_days"] = (cutoff_dt - cust_df["first_purchase_dt"]).dt.days
cust_df["purchases_per_90d"] = cust_df["n_purchases"] / (cust_df["tenure_days"] / 90.0).replace(0, np.nan)
cust_df["recency_days"] = cust_df["days_since_last"]
cust_df["monetary_mean"] = cust_df.get("ValorNeto_num_mean", np.nan)
cust_df["monetary_sum"] = cust_df.get("ValorNeto_num_sum", np.nan)

# Limpieza final de columnas temporales
cust_df = cust_df.drop(columns=['last_purchase_dt', 'first_purchase_dt'], errors='ignore') # Se eliminan las columnas de fecha datetime

cust_df = cust_df.reset_index().rename(columns={key_col: "cliente_id"})
cust_df.head()

,cliente_id,n_purchases,ValorNeto_num_sum,ValorNeto_num_mean,ValorNeto_num_std,ValorNeto_num_median,Vendedor_nunique,Oficina_nunique,Localidad_nunique,Localidad_Moda,...,Nombre1_Moda,mean_gap,dyn_threshold,days_since_last,churn,tenure_days,purchases_per_90d,recency_days,monetary_mean,monetary_sum
0,5,1817,1.403819e+09,772602.559774,729299.429540,690083.880,1,1,1,PALMIRA,...,GONZALEZ MARIA LAURA,0.350220,15.0,0,0,636,257.122642,0,772602.559774,1.403819e+09
1,8,2122,1.717844e+09,809539.847743,767350.064752,654888.945,1,1,1,SAN MARTIN,...,"VALESTRA, MARIO ALBERTO",0.300330,15.0,0,0,637,299.811617,0,809539.847743,1.717844e+09
2,9,1114,4.824796e+08,433105.589632,350738.538884,387101.125,1,1,1,GODOY CRUZ,...,GOÑI FLORES JOSE IGNACIO,0.566038,15.0,0,0,630,159.142857,0,433105.589632,4.824796e+08
3,20,1640,1.132321e+09,690439.607409,658398.985418,604659.995,1,1,1,RIVADAVIA,...,"RINI, LUIS ARTURO",0.388652,15.0,0,0,637,231.711146,0,690439.607409,1.132321e+09
4,23,2028,3.681706e+08,181543.700192,244441.608950,82389.805,1,1,1,PALMIRA,...,"MAÑUECO, ADRIANA LUCIA",0.314258,15.0,0,0,637,286.530612,0,181543.700192,3.681706e+08


In [52]:
import pandas as pd
import numpy as np

# --- 1. Definición del nombre del archivo ---
DATA_FILENAME = 'cust_df_final_for_streamlit.csv'

# --- 2. Preparación del DataFrame (Asegurar que solo tenga las columnas correctas) ---

# NOTA: Asegúrate de que 'cust_df' está definido en tu entorno.
# Si 'cust_df' incluye columnas intermedias (`days_since_last`, `dyn_threshold`),
# es mejor excluirlas antes de guardar, aunque Streamlit solo usará las columnas de features.

# Si quieres ser muy estricto con las columnas:
# 1. Obtén la lista de features usadas (X_cols) y añade 'cliente_id'
#    Asumimos que X_cols y el target_col ('churn') están definidos en celdas previas.
#
#    cols_to_keep = ['cliente_id'] + X_cols + ['churn'] 
#    df_to_save = cust_df[cols_to_keep]

# Si asumes que cust_df está limpio, simplemente lo guardamos:
df_to_save = cust_df.copy()

# --- 3. Limpieza y Conversión ---

# Convertir la columna 'cliente_id' a string para asegurar consistencia en Streamlit.
if 'cliente_id' in df_to_save.columns:
    df_to_save['cliente_id'] = df_to_save['cliente_id'].astype(str)

# Rellenar cualquier NaN que pudiera haber quedado. Streamlit lo necesita limpio.
# Usamos el mismo método que en el entrenamiento (mediana).
# Esto es crucial para asegurar que el archivo guardado es usable.
print("Rellenando NaN con la mediana antes de guardar...")
df_to_save = df_to_save.fillna(df_to_save.median(numeric_only=True))


# --- 4. Guardar el DataFrame ---
try:
    df_to_save.to_csv(DATA_FILENAME, index=False)
    print(f"\n✅ DataFrame '{DATA_FILENAME}' guardado exitosamente.")
    print(f"Número de filas: {df_to_save.shape[0]}, Número de columnas: {df_to_save.shape[1]}")
    print("Este archivo debe ser utilizado en la aplicación Streamlit.")
except Exception as e:
    print(f"\n⚠️ Error al guardar el DataFrame: {e}")

Rellenando NaN con la mediana antes de guardar...

✅ DataFrame 'cust_df_final_for_streamlit.csv' guardado exitosamente.
Número de filas: 1145, Número de columnas: 21
Este archivo debe ser utilizado en la aplicación Streamlit.


In [53]:
# Definimos la columna objetivo
target_col = "churn"

# División entrenamiento y test 
X = cust_df.select_dtypes(include=[np.number]).drop(columns=["churn"], errors="ignore")
y = cust_df[target_col].astype(int)

X = X.fillna(X.median(numeric_only=True))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE
)

# Preprocesamiento (escalado) y definición de modelos
preprocess = ColumnTransformer([("num", StandardScaler(), X.columns)], remainder="drop")

models = {
    "Logistic Regression": LogisticRegression(max_iter=500),
    "Random Forest": RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE),
    "HistGradientBoosting": HistGradientBoostingClassifier(random_state=RANDOM_STATE),
}

# Validación cruzada estratificada y evaluación de modelos

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# Pipeline de entrenamiento y evaluación por modelo 
results = {}
for name, model in models.items():
    pipe = Pipeline([("prep", preprocess), ("clf", model)])
    scores = cross_validate(pipe, X_train, y_train, scoring="roc_auc", cv=cv, n_jobs=-1)
    results[name] = scores["test_score"].mean()

pd.DataFrame.from_dict(results, orient="index", columns=["ROC_AUC"]).sort_values(by="ROC_AUC", ascending=False)


,ROC_AUC
HistGradientBoosting,0.998617
Logistic Regression,0.998490
Random Forest,0.996399


In [54]:
# Rehacemos el entrenamiento y la comparación de modelos excluyendo variables con leakage
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier

RANDOM_STATE = 42

# Construir X,y SI NO ESTUVIESEN ya definidos (caso autosuficiente):
if 'cust_df' in globals():
    target_col = 'churn'
    candidate_num = [c for c in cust_df.columns 
                     if c not in ['cliente_id','churn','last_purchase_dt','first_purchase_dt'] 
                     and np.issubdtype(cust_df[c].dtype, np.number)]
    X_all = cust_df[candidate_num].copy()
    y_all = cust_df[target_col].astype(int).copy()
else:
    X_all = X.copy()
    y_all = y.copy()

# Quitar variables con leakage (derivadas del label)
leaky_cols = [c for c in ['days_since_last','dyn_threshold','mean_gap','recency_days'] if c in X_all.columns]
X_safe = X_all.drop(columns=leaky_cols)

# Imputación simple y split
X_safe = X_safe.replace([np.inf, -np.inf], np.nan)
X_safe = X_safe.fillna(X_safe.median(numeric_only=True))

X_train, X_test, y_train, y_test = train_test_split(
    X_safe, y_all, test_size=0.2, stratify=y_all, random_state=RANDOM_STATE
)

# Preprocesamiento + modelos
preprocess = ColumnTransformer([("num", StandardScaler(), X_safe.columns)], remainder="drop")

models_safe = {
    "Logistic Regression": LogisticRegression(max_iter=500),
    "Random Forest": RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE),
    "HistGradientBoosting": HistGradientBoostingClassifier(random_state=RANDOM_STATE)
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# Definición de pipeline y evaluación
rows = []
for name, model in models_safe.items():
    pipe = Pipeline([("prep", preprocess), ("clf", model)])
    scores = cross_validate(
        pipe, X_train, y_train,
        scoring=["roc_auc", "f1", "precision", "recall", "accuracy"],
        cv=cv, n_jobs=-1, return_train_score=True
    )
    rows.append({
        "model": name,
        "roc_auc_mean": scores["test_roc_auc"].mean(),
        "f1_mean": scores["test_f1"].mean(),
        "precision_mean": scores["test_precision"].mean(),
        "recall_mean": scores["test_recall"].mean(),
        "accuracy_mean": scores["test_accuracy"].mean(),
        "roc_auc_train_mean": scores["train_roc_auc"].mean()
    })

comp_safe = pd.DataFrame(rows).sort_values("roc_auc_mean", ascending=False)
comp_safe


/Users/facundo/Documents/proyecto/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


,model,roc_auc_mean,f1_mean,precision_mean,recall_mean,accuracy_mean,roc_auc_train_mean
1,Random Forest,0.900363,0.523939,0.699937,0.436190,0.913744,1.000000
2,HistGradientBoosting,0.896297,0.491138,0.587357,0.436190,0.901746,1.000000
0,Logistic Regression,0.885773,0.101507,0.383333,0.058571,0.887568,0.887267


In [55]:
# ==============================================================================
# 0. DEFINICIONES NECESARIAS (Asumimos que las librerías están cargadas)
# ==============================================================================
# --- Importaciones (Se mantienen) ---
from sklearn.model_selection import StratifiedKFold, train_test_split, RandomizedSearchCV, cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from scipy.stats import randint, uniform
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support, roc_auc_score,
    brier_score_loss, classification_report, confusion_matrix, f1_score
)
import pandas as pd 
import numpy as np

# 1. Definir Constantes y Objetos
RANDOM_STATE = 42
# Se asume que X_train está definido
preprocess = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X_train.columns)
    ],
    remainder='passthrough'
)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE) 

# ----------------------------------------------------------------------------
# 🌟 MODIFICACIÓN CLAVE 1: FORZAR LA SELECCIÓN DE RANDOM FOREST
# (Basado en la tabla adjunta donde tiene el mejor 'roc_auc_mean' y 'roc_auc_train_mean')
# ----------------------------------------------------------------------------
best_name = "Random Forest"
print(f"Modelo seleccionado para optimización: {best_name}")


# ==============================================================================
# BLOQUE DE OPTIMIZACIÓN DE MODELO
# ==============================================================================

if best_name == "Random Forest":
    # 🌟 MODIFICACIÓN CLAVE 2: SE USA EL CLASIFICADOR Y PARAMETROS DE RANDOM FOREST
    base = RandomForestClassifier(random_state=RANDOM_STATE)
    
    # Parámetros para Random Forest. Limitando max_depth para ayudar a mitigar overfitting
    param_dist = {
        "clf__n_estimators": randint(300, 1000), # Rango ajustado
        "clf__max_depth": randint(3, 16),        # Limitando profundidad para regularización
        "clf__min_samples_split": randint(2, 10),
        "clf__min_samples_leaf": randint(1, 5),  # Valores bajos para mejor rendimiento
        "clf__max_features": ["sqrt", "log2", None],
    }
    n_iter_search = 50
    
elif best_name == "HistGradientBoosting":
    base = HistGradientBoostingClassifier(random_state=RANDOM_STATE)
    param_dist = {
        "clf__max_depth": randint(3, 12), 
        "clf__learning_rate": uniform(0.01, 0.1), 
        "clf__max_leaf_nodes": randint(15, 63),
        "clf__l2_regularization": uniform(0.0, 2.0),
    }
    n_iter_search = 50 
else:
    base = LogisticRegression(max_iter=500)
    param_dist = {"clf__C": uniform(0.01, 10.0)}
    n_iter_search = 50

tune_pipe = Pipeline([("prep", preprocess), ("clf", base)])
search = RandomizedSearchCV(
    estimator=tune_pipe, 
    param_distributions=param_dist,
    n_iter=n_iter_search, 
    # 🌟 Mantenemos 'f1' como métrica de scoring para optimizar el equilibrio de clases
    scoring="f1", 
    cv=cv, 
    n_jobs=-1, 
    random_state=RANDOM_STATE, 
    verbose=1
)
search.fit(X_train, y_train)

best_pipe = search.best_estimator_

# Evaluación exhaustiva en TEST

# Probabilidades para AUC, Brier y ajuste de umbral
try:
    y_proba = best_pipe.predict_proba(X_test)[:, 1]
except Exception:
    from sklearn.utils.extmath import softmax
    dec = best_pipe.decision_function(X_test)
    y_proba = (dec - dec.min()) / (dec.max() - dec.min() + 1e-9)

# -------------------------------------------------------------
# 🌟 AJUSTE DEL UMBRAL PARA ENCONTRAR EL EQUILIBRIO (0.25)
# -------------------------------------------------------------
# Mantenemos el umbral bajo para favorecer la reducción de Falsos Negativos.
threshold = 0.25 
print(f"**Ajuste de Umbral**: Buscando el equilibrio con un umbral de {threshold}")

y_pred = (y_proba >= threshold).astype(int)
# -------------------------------------------------------------


# Métricas basadas en el nuevo umbral (Resto del código de evaluación)
acc = accuracy_score(y_test, y_pred)
prec, rec, f1, _ = precision_recall_fscore_support(y_test, y_pred, average="binary", zero_division=0)
auc = roc_auc_score(y_test, y_proba) 
brier = brier_score_loss(y_test, y_proba)       
rmse_prob = np.sqrt(brier)                       

print("== Test metrics (Umbral ajustado a %.2f) ==" % threshold)
print(f"ROC-AUC : {auc:.4f}")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}  Recall: {rec:.4f}  F1: {f1:.4f}")
print(f"Brier score (MSE probas): {brier:.4f}  |  RMSE probas: {rmse_prob:.4f}")

print("\nClassification report:")
print(classification_report(y_test, y_pred, digits=3))
print("Confusion matrix (FN=15 / FP=10. Debería mejorar el equilibrio):\n", confusion_matrix(y_test, y_pred))

# Diagnóstico de Over/Under-fitting con CV en TRAIN
cv_diag = cross_validate(
    best_pipe,
    X_train,
    y_train,
    scoring="roc_auc",
    cv=cv,
    n_jobs=-1,
    return_train_score=True
)

keys = list(cv_diag.keys())
print("Keys disponibles en cross_validate:", keys)

train_key = next((k for k in keys if "train" in k), None)
test_key = next((k for k in keys if "test" in k), None)

train_auc = np.mean(cv_diag[train_key])
valid_auc = np.mean(cv_diag[test_key])

print("\n== Diagnóstico CV (ROC-AUC) ==")
print(f"Train AUC mean : {train_auc:.4f}")
print(f"Valid AUC mean : {valid_auc:.4f}")

gap = train_auc - valid_auc

if gap > 0.03:
    print("\n Posible OVERFITTING (train >> valid)")
elif valid_auc < 0.65:
    print("\n Posible UNDERFITTING (validación baja).")
else:
    print("\nAjuste razonable (sin señales fuertes de over/under-fitting).")

Modelo seleccionado para optimización: Random Forest
Fitting 5 folds for each of 50 candidates, totalling 250 fits
**Ajuste de Umbral**: Buscando el equilibrio con un umbral de 0.25
== Test metrics (Umbral ajustado a 0.25) ==
ROC-AUC : 0.9178
Accuracy: 0.8821
Precision: 0.4839  Recall: 0.5769  F1: 0.5263
Brier score (MSE probas): 0.0612  |  RMSE probas: 0.2473

Classification report:
              precision    recall  f1-score   support

           0      0.944     0.921     0.933       203
           1      0.484     0.577     0.526        26

    accuracy                          0.882       229
   macro avg      0.714     0.749     0.729       229
weighted avg      0.892     0.882     0.887       229

Confusion matrix (FN=15 / FP=10. Debería mejorar el equilibrio):
 [[187  16]
 [ 11  15]]
Keys disponibles en cross_validate: ['fit_time', 'score_time', 'test_score', 'train_score']

== Diagnóstico CV (ROC-AUC) ==
Train AUC mean : 0.9956
Valid AUC mean : 0.9067

 Posible OVERFITTING (tr

In [56]:
import joblib
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline # Necesitas definir best_pipe aquí

# Asegúrate de que 'best_pipe' es tu modelo final entrenado
# y 'X_train.columns' contiene los nombres de las features usadas.

# 1. Guardar el modelo Pipeline (que incluye preprocesamiento)
joblib.dump(best_pipe, 'modelo_churn_final.joblib')

# 2. Guardar la lista de columnas (features) usadas por el modelo. Esto es CRÍTICO.
feature_names = X_train.columns.tolist()
joblib.dump(feature_names, 'features_list.joblib')

print("Modelo y lista de features guardados exitosamente.")

Modelo y lista de features guardados exitosamente.
